In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, when, sum

In [2]:
spark = SparkSession.builder.appName("L3Q3").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/11 14:29:29 WARN Utils: Your hostname, dbl-23, resolves to a loopback address: 127.0.1.1; using 172.16.58.182 instead (on interface eno1)
25/08/11 14:29:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/11 14:29:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df1 = spark.read.csv('datasets/jaccard_similarity/part-00000-1bb539b7-39d7-48af-9cf7-318f538e7ec8-c000.csv', header=True)
df2 = spark.read.csv('datasets/jaccard_similarity/part-00001-1bb539b7-39d7-48af-9cf7-318f538e7ec8-c000.csv', header=True)

df = df1.union(df2)

In [20]:
df = df.withColumn('label', when(col('id_1') == col('id_2'), lit(1)).otherwise(lit(0)))
df = df.withColumn('predicted_label', when(col('jaccard_similarity') > 0.5, lit(1)).otherwise(lit(0)))

In [22]:
df = df.withColumn("TP", when((col("label") == 1) & (col("predicted_label") == 1), 1).otherwise(0))
df = df.withColumn("FP", when((col("label") == 0) & (col("predicted_label") == 1), 1).otherwise(0))
df = df.withColumn("TN", when((col("label") == 0) & (col("predicted_label") == 0), 1).otherwise(0))
df = df.withColumn("FN", when((col("label") == 1) & (col("predicted_label") == 0), 1).otherwise(0))

In [35]:
TP = df.agg({"TP":"sum"}).collect()[0]['sum(TP)']
FP = df.agg({"FP":"sum"}).collect()[0]['sum(FP)']
TP = df.agg({"TN":"sum"}).collect()[0]['sum(TN)']
FN = df.agg({"FN":"sum"}).collect()[0]['sum(FN)']

TP, FP, TP, FN

(26467214, 4639184, 26467214, 0)

In [38]:
precision = TP / (TP + FP)
recall = TP / (TP + FN)
F1 = (2*precision*recall) / (precision + recall)

print("Precision: ", precision, "Recall: ", recall, "F1: ", F1)

Precision:  0.8508607779017037 Recall:  1.0 F1:  0.9194216961756716


In [37]:
spark.stop()